In [19]:
using QuantumOptics
using LinearAlgebra

In [22]:
function Chern(Nx, Ny, t, p, q, N1, N2, PN)
    
    function KM_Hamiltonian(Nx, Ny, t, p, q, Tx, Ty)
        
        alpha = p/q

        xy = vcat(([y x] for x in 0:Nx-1 for y in 0:Ny-1)...)
        
        H = zeros(Complex{Float64}, Nx*Ny,Nx*Ny)
        
        for j in 1:Nx*Ny
            for k in 1:Nx*Ny
                if j!=k
                    
                    x_j = xy[j, 1] 
                    y_j = xy[j, 2] 
                    x_k = xy[k, 1]
                    y_k = xy[k, 2]
        
                    z_j = x_j + 1im*y_j
                    z_k = x_k + 1im*y_k
        
                    for n in range(-1,1)
                        for m in range(-1,1)
        
                            R = n * Nx + 1im * m * Ny
                            z = (z_k - z_j) + R 
                            G = (-1) ^ ( real(z) + imag(z) + real(z) * imag(z) )
        
                            beta = (pi/2) * ( (z_j * conj(z) ) - (conj(z_j) * z) )*alpha
                            W = t * G * exp( -pi/2 * (1-alpha) * (abs(z)^2) )
                            J = W * exp(beta) * exp( pi/2 * ( (z_j * conj(R) ) - (conj(z_j) * R) ) * alpha)
                            T = exp( 1im * real(R) / Nx * Tx + 1im * imag(R) /Ny * Ty)
        
                            H[j,k] += J*T
        
                        end
                    end
                    
                end
            end
        end
    
    return H
    end

    N = Nx * Ny
   
    function KM_Mb(Op, N, PN)
        
        basis = NLevelBasis(N) 
        states = bosonstates(basis, PN)
        basis_mb = ManyBodyBasis(basis, states)
    
        H = SparseOperator(basis_mb) 
        
        for m in 1:N
            for n in 1:N
                H += Op[m,n] * transition(basis_mb, m, n)
            end
        end
        
        return H
    
    end


    dx = 2*pi/N1
    dy = dx
    Tx = collect(range(start=0, stop=2*pi-dx, step=dx))
    Ty = collect(range(start=0, stop=2*pi-dy, step=dy))

    
    function Chern(Nx, Ny, t, p, q, Tx, Ty, dy, dx, PN, n1, n2)
        
        C = 0
        
        for tx in range(start=1, stop=length(Tx))
            for ty in range(start=1, stop=length(Ty))
    
                Op = KM_Hamiltonian(Nx, Ny, t, p, q, Tx[tx], Ty[ty])
                Op_Mb = KM_Mb(Op, N, PN)
                E1, U1 = eigen(dense(Op_Mb).data)
                U1 = U1[:,n1:n2]
    
                Op = KM_Hamiltonian(Nx, Ny, t, p, q, Tx[tx]+dx, Ty[ty])
                Op_Mb = KM_Mb(Op, N, PN)
                E2, U2 = eigen(dense(Op_Mb).data)
                U2 = U2[:,n1:n2]
    
                Op = KM_Hamiltonian(Nx, Ny, t, p, q, Tx[tx], Ty[ty]+dy)
                Op_Mb = KM_Mb(Op, N, PN)
                E3, U3 = eigen(dense(Op_Mb).data)
                U3 = U3[:,n1:n2]
    
                Op = KM_Hamiltonian(Nx, Ny, t, p, q, Tx[tx]+dx, Ty[ty]+dy)
                Op_Mb = KM_Mb(Op, N, PN)
                E4, U4 = eigen(dense(Op_Mb).data)
                U4 = U4[:,n1:n2]
    
                S1 = det(U1' * U2)
                S1 = S1 / abs(S1)
                
                S2 = det(U2' * U4)
                S2 = S2 / abs(S2)
                
                S3 = det(U3' * U4)
                S3 = S3 / abs(S3)
                
                S4 = det(U1' * U3)
                S4 = S4 / abs(S4)
    
                F12 = log(S1 * S2 * 1/S3 * 1/S4)
    
                C += F12
    
            end
        end
    
        return 1 / (2*pi*1im) * C
    end
    
    Cherns = []
    states = bosonstates(NLevelBasis(N), PN)
    for i in 1:Int(length(states)/Nx):length(states)
        push!(Cherns,round(Chern(Nx, Ny, t, p, q, Tx, Ty, dy, dx, PN, i, i + (Int(length(states) / Nx)) - 1)))
    end
        
    return Cherns
end

Chern (generic function with 1 method)

In [24]:
Nx = 5
Ny = 5
t = 1
p = 2
q = Ny
N1 = 10
N2 = 10
PN = 2
Chern(Nx, Ny, t, p, q, N1, N2, PN)

5-element Vector{Any}:
 -3.0 - 0.0im
  1.0 + 0.0im
  3.0 - 0.0im
 -1.0 - 0.0im
 -4.0 - 0.0im